In [1]:
{-# LANGUAGE ScopedTypeVariables, TypeOperators #-}
import Data.Manifold
import Data.Manifold.TreeCover
import Data.Manifold.Web
import Data.Manifold.Shade
import Data.Manifold.Types
import Data.Function.Affine
import Math.LinearMap.Category
import Data.VectorSpace
import Linear (V2(..), _x, _y)

In [2]:
test x = x`seq`putStrLn "Ok"

In [3]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' ℝ -> Shade (Colour ℝ)
colourscheme = f `seq` \(Shade' y e) -> f (Shade y $ dualNorm e)
 where Just (f :: Shade ℝ -> Shade (Colour ℝ))
          = rangeWithinVertices (1, maxc)
                              [ (0, black) ]
       Just maxc = toInterior (grey :: Colour ℝ)
test colourscheme

colourscheme_nabla :: Shade' (V2 ℝ+>ℝ) -> Shade (Colour ℝ)
colourscheme_nabla = f `seq` \(Shade' y e) -> f (Shade y $ dualNorm e)
 where Just (f :: Shade (V2 ℝ+>ℝ) -> Shade (Colour ℝ))
          = rangeWithinVertices (zeroV, neutral)
                              [ (V2 2 0-+|>1, red)
                              , (V2 0 2-+|>1, green) ]
       Just neutral = toInterior (grey :: Colour ℝ)
test colourscheme_nabla

Ok

Ok

In [4]:
gaußianPeak :: [ℝ²] -> ℝ -> PointsWeb ℝ² (Shade' ℝ)
gaußianPeak ps δy = fromWebNodes euclideanMetric
       [ (xy, exp (-2*magnitudeSq xy) |±|[δy]) | xy <- ps ]

In [7]:
δ = 0.01
grid = [V2 x y | x<-[-2,-1.8..2], y<-[-1,-0.8..2]]

plotWindow [plot . fmap colourscheme $ gaußianPeak grid δ ]
plotWindow [plot . fmap colourscheme_nabla . differentiateUncertainWebFunction
              $ gaußianPeak grid δ ]
plotWindow [plot . fmap (colourscheme_nabla . projectShade (lensEmbedding (1 ∂id/∂_x)))
   . differentiateUncertainWebFunction . differentiateUncertainWebFunction
       $ gaußianPeak grid δ ]
plotWindow [plot . fmap (colourscheme_nabla . projectShade (lensEmbedding ((1 ∂id/∂_x)/∂id)))
   . differentiateUncertainWebFunction . differentiateUncertainWebFunction
       $ gaußianPeak grid δ ]

GraphWindowSpecR2{lBound=-2.666666666666667, rBound=2.666666666666666, bBound=-1.4999999999999998, tBound=2.4999999999999987, xResolution=640, yResolution=480}

GraphWindowSpecR2{lBound=-2.666666666666666, rBound=2.6666666666666656, bBound=-1.4999999999999998, tBound=2.499999999999999, xResolution=640, yResolution=480}

GraphWindowSpecR2{lBound=-2.666666666666666, rBound=2.6666666666666656, bBound=-1.4999999999999993, tBound=2.4999999999999987, xResolution=640, yResolution=480}

GraphWindowSpecR2{lBound=-2.666666666666666, rBound=2.666666666666665, bBound=-1.4999999999999996, tBound=2.499999999999999, xResolution=640, yResolution=480}